#**Exploratory data analysis**

##**Data preprocessing** </br>
Please refer to section 2.a, titled 'Data preprocessing,' in the README.MD for the code related to transferring data into the database </br>

##**Data Preprocessing**</br>
Please refer to section 2.b in the README.MD for the code where extensive data cleaning is performed using SQL queries instead of Python. In Python, we solely rely on SQL queries to retrieve the already cleaned data. </br>.

In [2]:
#Libraries
import sqlite3
import pandas as pd
import os
import matplotlib
import numpy as np

In [4]:
con = sqlite3.connect('mydatabase.db')  #conneting to database 

In [8]:
df_box_score_query = """
WITH TeamAvTeamB AS (
    SELECT
        FixtureKey,
        SUBSTR(FixtureKey, 1, LENGTH(FixtureKey) - 12) AS TeamAvTeamB
    FROM
        box_scores
)
SELECT
    TeamAvTeamB.FixtureKey AS FixtureKey, 
    TRIM(CASE
            WHEN Team = 1 THEN SUBSTR(TeamAvTeamB, 1, INSTR(TeamAvTeamB, 'v') - 1)
            WHEN Team = 2 THEN SUBSTR(TeamAvTeamB, INSTR(TeamAvTeamB, 'v') + 1)
            ELSE NULL -- Handle other cases if needed
        END) AS TeamName,
    Team,X2PM,X2PA,X3PM,X3PA,FTM,FTA,ORB,DRB,AST,STL,BLK,TOV,PF
FROM
    TeamAvTeamB
    JOIN box_scores ON TeamAvTeamB.FixtureKey = box_scores.FixtureKey;"""

df_box_scores = pd.read_sql_query(df_box_score_query, con)

,FixtureKey,TeamName,Team,X2PM,X2PA,X3PM,X3PA,FTM,FTA,ORB,DRB,AST,STL,BLK,TOV,PF
0,LIPSCO v A PEAY 14-Jan-2023,LIPSCO,1,16,29,10,22,24,29,7,25,17,5,2,13,18
1,LIPSCO v A PEAY 14-Jan-2023,A PEAY,2,18,41,7,21,15,16,12,19,13,7,4,12,21
2,LIPSCO v A PEAY 14-Jan-2023,LIPSCO,1,16,29,10,22,24,29,7,25,17,5,2,13,18
3,LIPSCO v A PEAY 14-Jan-2023,A PEAY,2,18,41,7,21,15,16,12,19,13,7,4,12,21
4,QUENNC v A PEAY 29-Dec-2022,QUENNC,1,18,32,7,19,24,36,11,25,14,4,2,13,18
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26639,YSU v OAK 09-Feb-2022,OAK,2,18,32,5,23,20,25,10,24,13,7,1,14,16
26640,DET v YSU 12-Jan-2023,DET,1,26,44,6,23,9,12,11,26,12,3,2,5,16
26641,DET v YSU 12-Jan-2023,YSU,2,21,35,11,25,9,14,7,23,17,4,2,6,13
26642,DET v YSU 12-Jan-2023,DET,1,26,44,6,23,9,12,11,26,12,3,2,5,16


In [9]:
con.close()  #close the connnection